In [1]:
import tensorflow as tf
import scipy
import scipy.misc
import random
import os
from tensorflow.core.protobuf import saver_pb2

#load dataset
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66,200])/255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

#defining model

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.nn.leaky_relu(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) 


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 1000

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={ x:xs, y_: ys, keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch: 0, Step: 0, Loss: 6.49316
Epoch: 0, Step: 10, Loss: 6.54166
Epoch: 0, Step: 20, Loss: 6.64436
Epoch: 0, Step: 30, Loss: 6.4543
Epoch: 0, Step: 40, Loss: 6.1922
Model saved in file: ./save\model.ckpt
Epoch: 1, Step: 1000, Loss: 6.32876
Epoch: 1, Step: 1010, Loss: 5.99794
Epoch: 1, Step: 1020, Loss: 6.4862
Epoch: 1, Step: 1030, Loss: 5.83495
Epoch: 1, Step: 1040, Loss: 5.78761
Model saved in file: ./save\model.ckpt
Epoch: 2, Step: 2000, Loss: 6.37858
Epoch: 2, Step: 2010, Loss: 5.75158
Epoch: 2, Step: 2020, Loss: 5.68192
Epoch: 2, Step: 2030, Loss: 5.65367
Epoch: 2, Step: 2040, Loss: 5.68891
Model saved in file: ./save\model.ckpt
Epoch: 3, Step: 3000, Loss: 5.88897
Epoch: 3, Step: 3010, Loss: 6.17899
Epoch: 3, Step: 3020, Loss: 5.74998
Epoch: 3, Step: 3030, Loss: 5.67418
Epoch: 3, Step: 3040, Loss: 5.48404
Model saved in file: ./save\model.ckpt
Epoch: 4, Step: 4000, Loss: 5.95157
Epoch: 4, Step: 4010, Loss: 5.2726
Epoch: 4, Step: 4020, Loss: 5.22543
Epoch: 4, Step: 4030, Loss: 5.1

Epoch: 13, Step: 13010, Loss: 3.71793
Epoch: 13, Step: 13020, Loss: 3.65403
Epoch: 13, Step: 13030, Loss: 3.76757
Epoch: 13, Step: 13040, Loss: 3.88713
Model saved in file: ./save\model.ckpt
Epoch: 14, Step: 14000, Loss: 4.10274
Epoch: 14, Step: 14010, Loss: 3.85162
Epoch: 14, Step: 14020, Loss: 3.8292
Epoch: 14, Step: 14030, Loss: 3.73071
Epoch: 14, Step: 14040, Loss: 3.53411
Model saved in file: ./save\model.ckpt
Epoch: 15, Step: 15000, Loss: 4.00217
Epoch: 15, Step: 15010, Loss: 3.40833
Epoch: 15, Step: 15020, Loss: 3.37124
Epoch: 15, Step: 15030, Loss: 3.94019
Epoch: 15, Step: 15040, Loss: 3.35394
Model saved in file: ./save\model.ckpt
Epoch: 16, Step: 16000, Loss: 3.32881
Epoch: 16, Step: 16010, Loss: 3.31608
Epoch: 16, Step: 16020, Loss: 3.36188
Epoch: 16, Step: 16030, Loss: 3.54036
Epoch: 16, Step: 16040, Loss: 3.80984
Model saved in file: ./save\model.ckpt
Epoch: 17, Step: 17000, Loss: 3.54271
Epoch: 17, Step: 17010, Loss: 3.30542
Epoch: 17, Step: 17020, Loss: 3.35654
Epoch: 17

Model saved in file: ./save\model.ckpt
Epoch: 26, Step: 26000, Loss: 2.33205
Epoch: 26, Step: 26010, Loss: 2.28534
Epoch: 26, Step: 26020, Loss: 2.28238
Epoch: 26, Step: 26030, Loss: 2.9059
Epoch: 26, Step: 26040, Loss: 2.25792
Model saved in file: ./save\model.ckpt
Epoch: 27, Step: 27000, Loss: 2.22127
Epoch: 27, Step: 27010, Loss: 2.31641
Epoch: 27, Step: 27020, Loss: 2.41098
Epoch: 27, Step: 27030, Loss: 2.58322
Epoch: 27, Step: 27040, Loss: 2.54295
Model saved in file: ./save\model.ckpt
Epoch: 28, Step: 28000, Loss: 2.44393
Epoch: 28, Step: 28010, Loss: 2.37331
Epoch: 28, Step: 28020, Loss: 2.20659
Epoch: 28, Step: 28030, Loss: 2.66863
Epoch: 28, Step: 28040, Loss: 2.12131
Model saved in file: ./save\model.ckpt
Epoch: 29, Step: 29000, Loss: 2.0837
Epoch: 29, Step: 29010, Loss: 2.72165
Epoch: 29, Step: 29020, Loss: 2.07263
Epoch: 29, Step: 29030, Loss: 2.05882
Epoch: 29, Step: 29040, Loss: 2.06335
Model saved in file: ./save\model.ckpt
